# Frequent Itemset Mining

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
df2 = pd.read_excel('online_retail_II.xlsx')
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
# align column names
df = df.rename(columns={'UnitPrice': 'Price'})
df2 = df2.rename(columns={'Invoice': 'InvoiceNo'})

In [4]:
df2 = df2.rename(columns={'Customer ID': 'CustomerID'})

In [5]:
# make sure they're aligned
df.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True])

Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [6]:
data = pd.concat([df, df2])
data.shape

(1067370, 8)

In [7]:
data = data[data.InvoiceNo.notna() & (data.Quantity >= 0)]
data.shape

(1044420, 8)

In [8]:
# i'm running into memory problems, so let's delete the original dataframes
del df
del df2

Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [9]:
data = data[data.Country == 'United Kingdom']
data.shape

(961224, 8)

Question 1.3: What are the most popular 5 items?

There are two ways to consider "most popular": 

1) items that were ordered most commonly (appear in the greatest number of orders)

2) items that have the highest total quantity ordered

I show both methods here.

Also, somewhat confusingly, the "Stockcode" and "Description" columns have a different number of unique values, so I'm not sure which one to use for unique items. I'll just use Description for simplicity's sake.

In [10]:
print(len(data.StockCode.unique()))
print(len(data.Description.unique()))

4974
5440


In [11]:
# most common
data.Description.value_counts()[:5]

WHITE HANGING HEART T-LIGHT HOLDER    5574
REGENCY CAKESTAND 3 TIER              3577
JUMBO BAG RED RETROSPOT               3167
ASSORTED COLOUR BIRD ORNAMENT         2779
PARTY BUNTING                         2600
Name: Description, dtype: int64

In [12]:
# highest total quantity
data.groupby(['Description'])['Quantity'].sum().sort_values(ascending=False)[:5]

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     101464
WHITE HANGING HEART T-LIGHT HOLDER     89012
PAPER CRAFT , LITTLE BIRDIE            80995
MEDIUM CERAMIC TOP STORAGE JAR         77036
ASSORTED COLOUR BIRD ORNAMENT          76021
Name: Quantity, dtype: int64

Question 1.4: Filter down the data to include transaction that contain the top 20 items

Since I'm unsure which definition of "top" items to use, I've included both versions here below.

I tried running through the rest of the notebook with both versions. Only the second of the two resulted in any association rules with a support threshold of 0.07. So, I'll use that. 

In [33]:
# top20 = list(data.Description.value_counts()[:20].index)
top20 = list(data.groupby(['Description'])['Quantity'].sum().sort_values(ascending=False).index[:20])

In [34]:
filt = data.groupby('InvoiceNo').filter(lambda x: x.Description.isin(top20).any())
filt.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

The method I have chosen solves both questions 2.1 and 2.2 at the same time (MultiLabelBinarizer gives only 0 and 1).

In [35]:
#Create the "basket"
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
inds = []
product_lists = []
for name, group in filt.groupby('InvoiceNo'):
    inds.append(name)
    product_lists.append(group['Description'])
basket = pd.DataFrame(mlb.fit_transform(product_lists), index=inds, columns=mlb.classes_)

In [36]:
#Check to make sure you did it right
basket

,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,CAMPHOR WOOD PORTOBELLO MUSHROOM,CHERRY BLOSSOM DECORATIVE FLASK,DOLLY GIRL BEAKER,...,ZINC POLICE BOX LANTERN,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489442,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581498,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
581538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581579,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [37]:
from mlxtend.frequent_patterns import apriori

itemsets = apriori(basket, min_support=.07, use_colnames=True)

Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

In [38]:
from mlxtend.frequent_patterns import association_rules

association_rules(itemsets, metric="confidence", min_threshold=0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(RED HANGING HEART T-LIGHT HOLDER),(WHITE HANGING HEART T-LIGHT HOLDER),0.086661,0.318526,0.071208,0.821678,2.579626,0.043604,3.821598
1,(WHITE HANGING HEART T-LIGHT HOLDER),(RED HANGING HEART T-LIGHT HOLDER),0.318526,0.086661,0.071208,0.223554,2.579626,0.043604,1.176307


Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.

It looks like, in general, the red and white hanging heart t-light holders are sold together. However, it looks like the confidence is higher when the red hanging heart version is the antecedent.

As such, it seems like the best opportunity is to promote the white hanging heart version to customers who have already purchased the red one. 

Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

I had some very minimal experience with frequent mining; nothing more than playing around with it in my spare time (never for a real application). It didn't work very well on the dataset I tried it on. It was very useful to see it in action here, on a dataset appropriate to the application.

I had some trouble with ambiguity in the questions for this assignment. I wasn't sure what specifically was meant by "most popular" items. 